In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')


In [3]:
# device 
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using MPS


In [ ]:
# config
LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# data
TRAIN_PATH = '../data/train.csv'
TEST_PATH = '../data/test_1.csv'
VAL_SPLIT = 0.15
RANDOM_SEED = 42

# model
MODEL_NAME = 'distilroberta-base'
MAX_LENGTH = 128
HIDDEN_SIZE = 768
CLASSIFIER_HIDDEN = 256
DROPOUT = 0.1
NUM_LABELS = 6

# training
BATCH_SIZE = 128
EPOCHS = 3
LEARNING_RATE = 2e-5
# WEIGHT_DECAY = 0.01
# WARMUP_RATIO = 0.05
MAX_GRAD_NORM = 1.0

# inference
THRESHOLD = 0.5

# paths
MODEL_SAVE_PATH = '../models/bert.pth'


In [6]:
# load data
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# split train into train/val
train_df, val_df = train_test_split(train_df, test_size=VAL_SPLIT, random_state=RANDOM_SEED, stratify=train_df['toxic'])

print(f"Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}\n")
print(f"Label distribution (train):")
print(train_df[LABEL_COLS].sum())


Train: 135,635 | Val: 23,936 | Test: 63,978

Label distribution (train):
toxic            13000
severe_toxic      1366
obscene           7170
threat             402
insult            6691
identity_hate     1190
dtype: int64


In [7]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")


Tokenizer loaded: RobertaTokenizerFast


In [8]:
# dataset class
class BertDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=MAX_LENGTH):
        self.texts = df['comment_text'].values
        self.labels = df[LABEL_COLS].values.astype(np.float32)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        labels = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(labels, dtype=torch.float32)
        }

train_dataset = BertDataset(train_df, tokenizer)
val_dataset = BertDataset(val_df, tokenizer)
test_dataset = BertDataset(test_df, tokenizer)

print(f"Datasets created: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")


Datasets created: Train=135635, Val=23936, Test=63978


In [9]:
# model class
class ToxicClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = AutoModel.from_pretrained(MODEL_NAME)
        self.classifier = nn.Sequential(
            nn.Linear(HIDDEN_SIZE, CLASSIFIER_HIDDEN),
            nn.ReLU(),
            nn.Dropout(DROPOUT),
            nn.Linear(CLASSIFIER_HIDDEN, NUM_LABELS)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(pooled_output)
        return logits

model = ToxicClassifier().to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


Model parameters: 82,316,806
Trainable parameters: 82,316,806


In [ ]:
# training setup
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

total_steps = len(train_loader) * EPOCHS
# warmup_steps = int(total_steps * WARMUP_RATIO)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
# optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
criterion = nn.BCEWithLogitsLoss()

print(f"Total steps: {total_steps}")
# print(f"Total steps: {total_steps} | Warmup steps: {warmup_steps}")


Total steps: 3180 | Warmup steps: 159


In [ ]:
# evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(probs)
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    preds_binary = (all_preds >= THRESHOLD).astype(int)
    
    # macro metrics
    macro_precision = precision_score(all_labels, preds_binary, average='macro', zero_division=0)
    macro_recall = recall_score(all_labels, preds_binary, average='macro', zero_division=0)
    macro_f1 = f1_score(all_labels, preds_binary, average='macro', zero_division=0)
    
    # AUC-PR (average precision) per label
    per_label_auc_pr = []
    for i in range(NUM_LABELS):
        try:
            auc_pr = average_precision_score(all_labels[:, i], all_preds[:, i])
            per_label_auc_pr.append(auc_pr)
        except:
            per_label_auc_pr.append(0.0)
    
    avg_loss = total_loss / len(dataloader)
    macro_auc_pr = np.mean(per_label_auc_pr)
    
    return {
        'loss': avg_loss,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'macro_auc_pr': macro_auc_pr,
        'per_label_auc_pr': per_label_auc_pr
    }


In [ ]:
# training loop
from typing import Any


best_macro_f1 = 0
history = {'train_loss': [], 'val_loss': [], 'val_macro_precision': [], 'val_macro_recall': [], 'val_macro_f1': [], 'val_macro_auc_pr': []}

print(f"Starting training...\n")

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    
    # training
    model.train()
    train_loss = 0
    progress_bar = tqdm[Any](train_loader, desc=f"Training")
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        optimizer.step()
        # scheduler.step()
        
        train_loss += loss.item()
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    avg_train_loss = train_loss / len(train_loader)
    
    # validation
    val_metrics = evaluate(model, val_loader, device)
    
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(val_metrics['loss'])
    history['val_macro_precision'].append(val_metrics['macro_precision'])
    history['val_macro_recall'].append(val_metrics['macro_recall'])
    history['val_macro_f1'].append(val_metrics['macro_f1'])
    history['val_macro_auc_pr'].append(val_metrics['macro_auc_pr'])
    
    print(f"\nTrain Loss: {avg_train_loss:.4f}")
    print(f"Val Loss: {val_metrics['loss']:.4f}")
    print(f"Val Macro Precision: {val_metrics['macro_precision']:.4f}")
    print(f"Val Macro Recall: {val_metrics['macro_recall']:.4f}")
    print(f"Val Macro F1: {val_metrics['macro_f1']:.4f}")
    print(f"Val Macro AUC-PR: {val_metrics['macro_auc_pr']:.4f}")
    
    # save best model
    if val_metrics['macro_f1'] > best_macro_f1:
        best_macro_f1 = val_metrics['macro_f1']
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"✓ Saved best model (Macro F1: {best_macro_f1:.4f})")

print("Training complete!")
print(f"Best Val Macro F1: {best_macro_f1:.4f}")


Starting training...

Epoch 1/3


Training:   0%|          | 0/1060 [00:00<?, ?it/s]

Training:   0%|          | 0/1060 [02:41<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# evaluate
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
test_metrics = evaluate(model, test_loader, device)

print("Test Set Results:")
print(f"Macro Precision: {test_metrics['macro_precision']:.4f}")
print(f"Macro Recall: {test_metrics['macro_recall']:.4f}")
print(f"Macro F1: {test_metrics['macro_f1']:.4f}")
print(f"Macro AUC-PR: {test_metrics['macro_auc_pr']:.4f}")
print("\nPer-label AUC-PR:")
for i, label in enumerate(LABEL_COLS):
    print(f"  {label:15s}: {test_metrics['per_label_auc_pr'][i]:.4f}")
